# Part 1 - Setup and Data Preparation
In this section, we will:
- Import necessary packages for this demonstration
- Load the data
- Ensure that qualitative predictor variables are of the *category* data type

In [ ]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Import 'numpy' and 'pandas' for working with numbers and data frames
import numpy as np
import pandas as pd

# Import 'pyplot' from 'matplotlib' and 'seaborn' for visualizations
from matplotlib import pyplot as plt
import seaborn as sns

# Import method for regression from 'statsmodels'
import statsmodels.formula.api as smf

# Import method for regression from 'sklearn'
from sklearn.linear_model import LinearRegression

# Import 'train_test_split' from 'sklearn' for train-validation-test split
from sklearn.model_selection import train_test_split

# Import 'mean_squared_error' from 'sklearn' for error computations
from sklearn.metrics import mean_squared_error

# Import method to compute VIFs
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Data source
# link = 'https://archive.ics.uci.edu/dataset/10/automobile'

In [ ]:
# Load the data and take a look at it
df = pd.read_csv('carprices.csv', index_col = 'car_ID')
df.head()

In [ ]:
# Load the data description and take a look at it
dd = pd.read_csv('carpricesdatadescription.csv', index_col = 'feature')
dd

In [ ]:
# Look at the specifics of the data frame
df.info()

Most features in this dataset are numeric, being either of the *int64* datatype or the *float64* datatype, but some features are of the *object* datatype. These are, in fact, strings. Typically, when conducting categorical analysis, it is better to convert these into the *category* datatype.

In [ ]:
# Convert qualitative predictors to the 'category' data type
categorical_columns = df.select_dtypes(include = 'object').columns
df[categorical_columns] = df[categorical_columns].astype('category')

In [ ]:
# Look at the specifics of the data frame
df.info()

**Note:** The *object* datatype entries are now of the *category* datatype. This helps in conducting categorical analysis more efficiently.

It is important to note that the *symboling* feature has been recorded as an integer but it is actually a categorical variable. We can understand this from the data description.

In [ ]:
# Convert the 'symboling' feature to the 'category' datatype
df['symboling'] = df['symboling'].astype('category')

In [ ]:
# Drop the car name as it won't be useful in the overall analysis
df.drop(labels = 'carname', axis = 1, inplace = True)

**Note:** Names, identifiers, and so on, are generally not considered as valid predictor variables.

In [ ]:
# Take a look at the data
df.head()

In [ ]:
# Check the shape and size of the data
df.shape

**Note:** This is, of course, a small dataset. For this demonstration, we will use a subset of the features. Learners may explore with the original dataset at a later time.

In [ ]:
# Retain a select number of columns for the purposes of this demonstration
df = df[['symboling',
         'carbody',
         'enginelocation',
         'carlength',
         'carwidth',
         'carheight',
         'curbweight',
         'horsepower',
         'citympg',
         'highwaympg',
         'price']]

In [ ]:
# Take a look at the data
df.head()

In [ ]:
# Check the shape and size of the data
df.shape

In [ ]:
# Look at the specifics of the data frame
df.info()

In [ ]:
# Store the categorical column names
categorical_columns = df.select_dtypes(include = 'category').columns

In [ ]:
# Split the data into input and output
X = df.drop(labels = 'price', axis = 1)
y = df['price']

**Note:** The target variable here is the price of the car.

In [ ]:
# Split the data into training and testing datasets
# Note: There's no validation dataset because we are not fine-tuning any of the models
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

**Note:** Fixing a random state value ensures reproducibility of the analysis.

In [ ]:
# Check the number of observations in the train and test datasets
print('Training dataset size: ', len(X_train))
print('Testing dataset size: ', len(X_test))

# Part 2 - Exploratory Data Analysis
In this section, we will conduct EDA on the dataset

In [ ]:
# Create count plots for the categorical variables
plt.figure(figsize = (8, 3))

fignum = 0
for featurename in categorical_columns:
    fignum = fignum + 1
    plt.subplot(1, 3, fignum)
    sns.countplot(data = df, x = featurename)
    plt.xticks(rotation = 45)

plt.tight_layout();

In [ ]:
# Drop the engine location feature as there is very high class imbalance
df.drop(labels = 'enginelocation', axis = 1, inplace = True)

In [ ]:
# Create a pair plot for the numerical features in the data set
# sns.pairplot(df);

In [ ]:
# Look at the correlation matrix for the numerical features in the dataset
df.corr().round(2)

**Note:** Correlation values can tell us about redundancies in the dataset. For example, here, there's a very high correlation between the car weight and the car length. So, we can consider dropping on of these features if need be.

In [ ]:
df_cat = pd.get_dummies(data = df, columns = ['symboling', 'carbody'], drop_first = False)

In [ ]:
# Look at the correlation matrix for all the features in the dataset
df_cat.corr().round(2)

**Note:** Interpreting correlation matrices with categorical features can be challenging.

# Part 3 - Multiple Linear Regression Models
In this section, we will train and evaluate linear regression models for the data

## Numerical Predictors - *statsmodels*

### Model 1

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'carlength'
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_1 = smf.ols('price ~ carlength', data = df)
lr_model_1 = lr_model_1.fit()
print(lr_model_1.summary())

### Model 2

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'carlength' and 'carwidth'
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_2 = smf.ols('price ~ carlength + carwidth', data = df)
lr_model_2 = lr_model_2.fit()
print(lr_model_2.summary())

### Model 3

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'carlength', 'carwidth' and 'carheight'
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_3 = smf.ols('price ~ carlength + carwidth + carheight', data = df)
lr_model_3 = lr_model_3.fit()
print(lr_model_3.summary())

### Model 4

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using all the numerical predictors
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_4 = smf.ols('price ~ carlength + carwidth + carheight + curbweight + horsepower + citympg + highwaympg', data = df)
lr_model_4 = lr_model_4.fit()
print(lr_model_4.summary())

**Note:** In general practice, we may begin by fitting a linear regression model using all the input features and then assessing which features are helpful and which are not.

The *carlength*, *carweight*, *citympg* and *highwaympg* features seem to be statistically insignificant. Let us keep this in mind as we go forward with the analysis.

### Model 5

In [ ]:
# Look at the correlation matrix for the numerical features in the dataset
df.corr().round(2)

In [ ]:
# Retain correlation values greater than or equal to 0.6
df.corr().round(2)[abs(df.corr()) >= 0.6]

We can try to choose combinations of features that are not correlated strongly with each other but are somewhat correlated with the target variable.

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using all the numerical predictors except 'highwaympg' and 'carlength'
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_5 = smf.ols('price ~ carwidth + carheight + curbweight + horsepower + citympg', data = df)
lr_model_5 = lr_model_5.fit()
print(lr_model_5.summary()) 

The *carheight* feature seems to be statistically insignificant.

### Model 6

In [ ]:
# Retain correlation values greater than or equal to 0.6
df.corr().round(2)[abs(df.corr()) >= 0.6]

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using all the numerical predictors except 'highwaympg', 'carlength' and 'carheight'
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_6 = smf.ols('price ~ carwidth + curbweight + horsepower + citympg', data = df)
lr_model_6 = lr_model_6.fit()
print(lr_model_6.summary())

Adjusted R-squared seems to have increased without any change to R-squared. This is a good thing.

### Model 7

In [ ]:
# Retain correlation values greater than or equal to 0.6
df.corr().round(2)[abs(df.corr()) >= 0.6]

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'curbweight', 'horsepower' and 'citympg'
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_7 = smf.ols('price ~ curbweight + horsepower + citympg', data = df)
lr_model_7 = lr_model_7.fit()
print(lr_model_7.summary())

This looks like a decent model.

### Model 8

### Numerical Predictors - VIF

In [ ]:
# Obtain the VIFs for the numerical features in the dataset
# Note: Using all the data for analytical purposes but when building models generally only training data is used
numerical_columns = ['carlength', 'carwidth', 'carheight', 'curbweight', 'horsepower', 'citympg', 'highwaympg']
vif_data = pd.DataFrame()
vif_data['feature'] = numerical_columns
vif_data['VIF'] = [variance_inflation_factor(df[numerical_columns].values, i) for i in range(len(df[numerical_columns].columns))]
vif_data.round(2)

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'curbweight' and 'horsepower'
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_8 = smf.ols('price ~ curbweight + horsepower', data = df)
lr_model_8 = lr_model_8.fit()
print(lr_model_8.summary())

This model has decent R-squared and no statistically insignificant features.

**Note:** When creating and observing linear regression models for the purpose of analysis of predictor impact on target variable, we may choose to use the complete dataset, but when building ML models for the purpose of deployment, only the training data is used for analytical and model-building tasks.

## Numerical Predictors - *sklearn*

### Model 1

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'carlength'
# Note: Using only the training data
lr_model_1 = LinearRegression()
lr_model_1 = lr_model_1.fit(X_train[['carlength']], y_train)

In [ ]:
# Look at the intercept and coefficient values
print('Intercept: ', lr_model_1.intercept_)
print('Coefficient: ', lr_model_1.coef_)

In [ ]:
# Obtain predictions on the testing set
lr_model_1.predict(X_test[['carlength']])

In [ ]:
# Look at the general predictive performance of the model
pd.DataFrame(index = X_test.index,
             data = {'Truths': y_test,
                     'Predictions': lr_model_1.predict(X_test[['carlength']])}).head()

In [ ]:
# Summarize the performance of the model on the test data using RMSE and MAPE
y_pred_lr_list = lr_model_1.predict(X_test[['carlength']])
rmse = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_lr_list))
mape = np.mean(np.abs(y_test - y_pred_lr_list) / y_test) * 100

rmse = np.round(rmse, 2)
mape = np.round(mape, 2)

performance_df = pd.DataFrame(index = [0],
                              data = {'Model': 'SLR model 1', 'RMSE': rmse, 'MAPE': mape})

performance_df.set_index(keys = 'Model', inplace = True)

performance_df

### Model 2

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'carlength' and 'carwidth'
# Note: Using only the training data
lr_model_2 = LinearRegression()
lr_model_2 = lr_model_2.fit(X_train[['carlength', 'carwidth']], y_train)

In [ ]:
# Look at the intercept and coefficient values
print('Intercept: ', lr_model_2.intercept_)
print('Coefficient: ', lr_model_2.coef_)

In [ ]:
# Obtain predictions on the testing set
lr_model_2.predict(X_test[['carlength', 'carwidth']])

In [ ]:
# Look at the general predictive performance of the model
pd.DataFrame(index = X_test.index,
             data = {'Truths': y_test,
                     'Predictions': lr_model_2.predict(X_test[['carlength', 'carwidth']])}).head()

In [ ]:
# Summarize the performance of the model on the test data using RMSE and MAPE
y_pred_lr_list = lr_model_2.predict(X_test[['carlength', 'carwidth']])
rmse = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_lr_list))
mape = np.mean(np.abs(y_test - y_pred_lr_list) / y_test) * 100

rmse = np.round(rmse, 2)
mape = np.round(mape, 2)

performance_df_temp = pd.DataFrame(index = [0],
                                   data = {'Model': 'MLR model 2', 'RMSE': rmse, 'MAPE': mape})

performance_df_temp.set_index(keys = 'Model', inplace = True)

performance_df = pd.concat(objs = [performance_df, performance_df_temp])

performance_df

### Models 1 through 8

In [ ]:
# Store a list of the model names
model_names = []
model_names.append('SLR model 1')
for i in np.arange(2, 9, 1):
    model_names.append('MLR model ' + str(i))
model_names

In [ ]:
# Store the different combinations of input features of the models in a list
model_inputs = [['carlength'],
                ['carlength', 'carwidth'],
                ['carlength', 'carwidth', 'carheight'],
                numerical_columns,
                ['carwidth', 'carheight', 'curbweight', 'horsepower', 'citympg'],
                ['carwidth', 'curbweight', 'horsepower', 'citympg'],
                ['curbweight', 'horsepower', 'citympg'],
                ['curbweight', 'horsepower']]

In [ ]:
# Create a data frame to store the names of these models and their predictive performance values
performance_df = pd.DataFrame(index = model_names, data = {'RMSE': None, 'MAPE': None})

In [ ]:
# Loop through these models by training, evaluating and storing the predictive performance values
i = -1
for model_input in model_inputs:
    i = i + 1
    temp_lr_model = LinearRegression()
    temp_lr_model = temp_lr_model.fit(X_train[model_input], y_train)
    y_pred_lr_list = temp_lr_model.predict(X_test[model_input])
    rmse = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_lr_list))
    mape = np.mean(np.abs(y_test - y_pred_lr_list) / y_test) * 100
    performance_df.loc[model_names[i], 'RMSE'] = np.round(rmse, 2)
    performance_df.loc[model_names[i], 'MAPE'] = np.round(mape, 2)
performance_df

It seems that the model that uses *curbweight*, *horsepower* and *citympg* to predict the price of the car is doing well.

## Numerical and Categorical Predictors - *statsmodels*

### Model 9

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using all the numerical predictors and the 'carbody' predictor
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_9 = smf.ols('price ~ carlength + carwidth + carheight + curbweight + horsepower + citympg + highwaympg + carbody', data = df)
lr_model_9 = lr_model_9.fit()
print(lr_model_9.summary())

### Model 10

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using all the predictors
# Note: Using all the data for analytical purposes but when building models generally only training data is used
lr_model_10 = smf.ols('price ~ carlength + carwidth + carheight + curbweight + horsepower + citympg + highwaympg + carbody + symboling', data = df)
lr_model_10 = lr_model_10.fit()
print(lr_model_10.summary())

**Note:** Though the same rules of interpretation may be applied in these cases as well, it may be challenging to understand the impact of categorical features in the predictive power of the model.

## Numerical and Categorical Predictors - *sklearn*

In [ ]:
# Obtain dummy variables for the categorical features in both the training and testing input data
X_train_dummies = pd.get_dummies(data = X_train, columns = ['carbody', 'symboling'], drop_first = True)
X_test_dummies = pd.get_dummies(data = X_test, columns = ['carbody', 'symboling'], drop_first = True)

### Model 11

In [ ]:
# Look at the correlation matrix for all the features in the dataset
df_cat.corr().round(2)

In [ ]:
# Retain correlation values greater than or equal to 0.6
df_cat.corr().round(2)[abs(df_cat.corr()) >= 0.6]

**Note:** The problem with categorical features and correlation values is that categorical variables have binary values for each level. The interpretability of the correlation coefficient in this case can be questioned.

### Numerical and Categorical Predictors - VIF

In [ ]:
# Obtain the VIFs for all the features in the dataset
# Note: Using all the data for analytical purposes but when building models generally only training data is used
df_dummies = pd.get_dummies(data = df, columns = ['carbody', 'symboling'], drop_first = True)
all_columns = df_dummies.drop('price', axis = 1).columns
vif_data = pd.DataFrame()
vif_data['feature'] = all_columns
vif_data['VIF'] = [variance_inflation_factor(df_dummies[all_columns].values, i) for i in range(len(df_dummies[all_columns].columns))]
vif_data.round(2)

**Note:** The categorical variables are interfering with the VIF values of the numerical variables.

In [ ]:
# View the features with VIFs less than 50
vif_data[vif_data['VIF'] < 50]

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using all the features with apparently low VIFs
# Note: Using only the training data
lr_model_11 = LinearRegression()
lr_model_11 = lr_model_11.fit(X_train_dummies[vif_data[vif_data['VIF'] < 50]['feature'].values], y_train)

In [ ]:
# Look at the predictive performance of this model
y_pred_lr_list = lr_model_11.predict(X_test_dummies[vif_data[vif_data['VIF'] < 50]['feature'].values])
rmse = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_lr_list))
mape = np.mean(np.abs(y_test - y_pred_lr_list) / y_test) * 100

rmse = np.round(rmse, 2)
mape = np.round(mape, 2)

print('RMSE = ', rmse)
print('MAPE = ', mape)

### Model 12

In [ ]:
# Retain correlation values greater than or equal to 0.6
df_cat.corr().round(2)[abs(df_cat.corr()) >= 0.6]

In [ ]:
# Look at the summary output of the analytical model with all the predictors
print(lr_model_10.summary())

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'curbweight', 'horsepower', 'citympg', 'carbody' and 'symboling'
# Note: Using only the training data
lr_model_12 = LinearRegression()
lr_model_12 = lr_model_12.fit(X_train_dummies[['curbweight', 'horsepower', 'citympg', 'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan', 'carbody_wagon', 'symboling_-1', 'symboling_0', 'symboling_1', 'symboling_2', 'symboling_3']], y_train)

In [ ]:
# Look at the predictive performance of this model
y_pred_lr_list = lr_model_12.predict(X_test_dummies[['curbweight', 'horsepower', 'citympg', 'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan', 'carbody_wagon', 'symboling_-1', 'symboling_0', 'symboling_1', 'symboling_2', 'symboling_3']])
rmse = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_lr_list))
mape = np.mean(np.abs(y_test - y_pred_lr_list) / y_test) * 100

rmse = np.round(rmse, 2)
mape = np.round(mape, 2)

print('RMSE = ', rmse)
print('MAPE = ', mape)

### Model 13

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'curbweight', 'horsepower', 'citympg' and 'symboling'
# Note: Using only the training data
lr_model_13 = LinearRegression()
lr_model_13 = lr_model_13.fit(X_train_dummies[['curbweight', 'horsepower', 'citympg', 'symboling_-1', 'symboling_0', 'symboling_1', 'symboling_2', 'symboling_3']], y_train)

In [ ]:
# Look at the predictive performance of this model
y_pred_lr_list = lr_model_13.predict(X_test_dummies[['curbweight', 'horsepower', 'citympg', 'symboling_-1', 'symboling_0', 'symboling_1', 'symboling_2', 'symboling_3']])
rmse = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_lr_list))
mape = np.mean(np.abs(y_test - y_pred_lr_list) / y_test) * 100

rmse = np.round(rmse, 2)
mape = np.round(mape, 2)

print('RMSE = ', rmse)
print('MAPE = ', mape)

### Model 14

In [ ]:
# Create and train a linear regression model for the data and view its summary
# Note: The objective is to predict 'price' using 'curbweight', 'horsepower', and 'symboling'
# Note: Using only the training data
lr_model_14 = LinearRegression()
lr_model_14 = lr_model_14.fit(X_train_dummies[['curbweight', 'horsepower', 'symboling_-1', 'symboling_0', 'symboling_1', 'symboling_2', 'symboling_3']], y_train)

In [ ]:
# Look at the predictive performance of this model
y_pred_lr_list = lr_model_14.predict(X_test_dummies[['curbweight', 'horsepower', 'symboling_-1', 'symboling_0', 'symboling_1', 'symboling_2', 'symboling_3']])
rmse = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_lr_list))
mape = np.mean(np.abs(y_test - y_pred_lr_list) / y_test) * 100

rmse = np.round(rmse, 2)
mape = np.round(mape, 2)

print('RMSE = ', rmse)
print('MAPE = ', mape)

It seems as though the model that uses *curbweight*, *horsepower*, *citympg*, *carbody* and *symboling* is doing well on the testing data.

In [ ]:
temp_model = LinearRegression()
temp_model = temp_model.fit(X_train_dummies[['curbweight', 'horsepower', 'citympg', 'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan', 'carbody_wagon']], y_train)
y_pred_lr_list = temp_model.predict(X_test_dummies[['curbweight', 'horsepower', 'citympg', 'carbody_hardtop', 'carbody_hatchback', 'carbody_sedan', 'carbody_wagon']])
rmse = np.sqrt(mean_squared_error(y_true = y_test, y_pred = y_pred_lr_list))
mape = np.mean(np.abs(y_test - y_pred_lr_list) / y_test) * 100

rmse = np.round(rmse, 2)
mape = np.round(mape, 2)

print('RMSE = ', rmse)
print('MAPE = ', mape)